# Encoder-Decoder Model

Exercise: Train an Encoder–Decoder model that can convert a date string from one format to another (e.g., from "April 22, 2019" to "2019-04-22").

## Data

Start by creating the dataset. We will use random days between 1000-01-01 and 9999-12-31:

In [1]:
from datetime import date
import numpy as np
import tensorflow as tf

In [2]:
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.strftime("%Y-%m-%d") for dt in dates]
    return x, y

Here are a few random dates, displayed in both the input format(x) and the target format(y):

In [3]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


Let's get the list of all possible characters in the inputs(所有可能的input，包括數字、空格、逗號、所有月份縮寫字母):

In [4]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

And here's the list of possible characters in the outputs(output只會有數字和-):

In [5]:
OUTPUT_CHARS = "0123456789-"

Let's write a function to convert a string to a list of character IDs

In [6]:
def date_str_to_ids(string_date, char = INPUT_CHARS):
    return [char.index(charachter) for charachter in string_date]

In [7]:
x_example[0]

'September 20, 7075'

In [8]:
date_str_to_ids(x_example[0])

[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [9]:
y_example[0]

'7075-09-20'

In [10]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [11]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)
    

In [12]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [38]:
X_train

<tf.Tensor: shape=(10000, 18), dtype=int32, numpy=
array([[20, 24, 32, ...,  3, 10,  8],
       [17, 21, 38, ...,  0,  0,  0],
       [16, 21, 30, ...,  6,  0,  0],
       ...,
       [16, 21, 30, ..., 11,  0,  0],
       [15, 24, 22, ...,  5,  8,  0],
       [16, 36, 28, ...,  0,  0,  0]])>

In [37]:
X_train[0]

<tf.Tensor: shape=(18,), dtype=int32, numpy=
array([20, 24, 32, 35, 24, 29, 22, 24, 33,  1,  5,  3,  2,  1, 10,  3, 10,
        8])>

In [13]:
Y_train

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[ 8,  1,  8, ..., 11,  3,  1],
       [ 9,  6,  8, ..., 11,  2,  6],
       [ 8,  2,  1, ..., 11,  2,  2],
       ...,
       [10,  8,  7, ..., 11,  4,  1],
       [ 2,  2,  3, ..., 11,  3,  8],
       [ 8,  9,  4, ..., 11,  3, 10]])>

In [14]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1])>

10000個data，每一個Y都是長度為10的sequence(ex. 2024-02-26)

In [17]:
Y_train.shape

TensorShape([10000, 10])

## First version: a very basic seq2seq model

Let's first try the simplest possible model: we feed in the input sequence, which first goes through the encoder (an embedding layer followed by a single LSTM layer), which outputs a vector, then it goes through a decoder (a single LSTM layer, followed by a dense output layer), which outputs a sequence of vectors, each representing the estimated probabilities for all possible output character.

Since the decoder expects a sequence as input, we repeat the vector (which is output by the encoder) as many times as the longest possible output sequence.

(The reason for +1 is to reserve one index for out-of-vocabulary (OOV) tokens. During training or inference, it's possible to encounter characters that were not seen during training, and having a dedicated index for OOV tokens allows the model to handle them gracefully, typically by assigning them a special embedding vector.)

In [20]:
embedding_size = 32
max_output_length = Y_train.shape[1] #10

np.random.seed(42)
tf.random.set_seed(42)

encoder = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(INPUT_CHARS) + 1,
                           output_dim=embedding_size, #可以tune
                           input_shape=[None]), #input長度不一
    tf.keras.layers.LSTM(128)
])

decoder = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax")
])

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.RepeatVector(max_output_length),
    decoder
])

optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))






Epoch 1/20
313/313 [==============================] - 10s 21ms/step - loss: 1.7121 - accuracy: 0.3930 - val_loss: 1.2735 - val_accuracy: 0.5448
Epoch 2/20
313/313 [==============================] - 6s 19ms/step - loss: 1.1166 - accuracy: 0.6046 - val_loss: 0.9476 - val_accuracy: 0.6557
Epoch 3/20
313/313 [==============================] - 6s 19ms/step - loss: 0.7864 - accuracy: 0.7056 - val_loss: 0.6655 - val_accuracy: 0.7395
Epoch 4/20
313/313 [==============================] - 6s 18ms/step - loss: 0.6388 - accuracy: 0.7531 - val_loss: 0.5267 - val_accuracy: 0.7890
Epoch 5/20
313/313 [==============================] - 6s 19ms/step - loss: 0.4224 - accuracy: 0.8295 - val_loss: 0.3562 - val_accuracy: 0.8554
Epoch 6/20
313/313 [==============================] - 6s 19ms/step - loss: 0.2878 - accuracy: 0.8910 - val_loss: 0.2375 - val_accuracy: 0.9158
Epoch 7/20
313/313 [==============================] - 6s 19ms/step - loss: 0.1887 - accuracy: 0.9410 - val_loss: 0.1419 - val_accuracy: 0.962

In [30]:
X_new = prepare_date_strs(["September 17, 2009", "July 14, 1789"])
new_ids = model.predict(X_new).argmax(axis=-1)
new_ids

1/1 [==============================] - 0s 46ms/step


array([[ 3,  1,  1, 10, 11,  1, 10, 11,  2,  8],
       [ 2,  8,  9, 10, 11,  1,  8, 11,  2,  5]], dtype=int64)

since the model was only trained on input strings of length 18 (which is the length of the longest date ex. September 17, 2009), it does not perform well if we try to use it to make predictions on shorter sequences(ex. May 02, 2020).

 We need to ensure that we always pass new testing sequences data of the same length as during training, using padding if necessary. Let's write a little helper function for that:

In [24]:
X_train.shape

TensorShape([10000, 18])

In [21]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X



* 0 at index 0 of the outer list [0, 0] indicates that no padding is added before or after the elements along the first dimension (typically the batch dimension).

* [0, max_input_length - X.shape[1]] specifies the padding along the second dimension (axis 1).
    
    * 0 at index 0 indicates no padding is added before the elements.

    * max_input_length - X.shape[1] at index 1 calculates the amount of padding needed to match max_input_length along the second dimension. X.shape[1] represents the current length of the second dimension of X, and subtracting it from max_input_length gives the amount of padding needed.

現在每一個sequence長度都為18:

In [23]:
prepare_date_strs_padded("May 02, 2020")

<tf.Tensor: shape=(12, 18), dtype=int32, numpy=
array([[17,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [38,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

We will need to be able to convert a sequence of character IDs to a readable string:

In [35]:
OUTPUT_CHARS
("?" + OUTPUT_CHARS)[2] #隨便舉例，方便了解下面code

'0123456789-'

In [40]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence]) #?代表以防有未知的字
            for sequence in ids]

ids_to_date_strs(new_ids)

['2009-09-17', '1789-07-14']

In [ ]:

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = model.predict(X).argmax(axis=-1)
    return ids_to_date_strs(ids)

In [33]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

1/1 [==============================] - 0s 46ms/step


['2020-05-02', '1789-07-14']

## Second version: feeding the shifted targets to the decoder (teacher forcing)

Instead of feeding the decoder a simple repetition of the encoder's output vector, we can feed it the target sequence, shifted by one time step to the right. This way, at each time step the decoder will know what the previous target character was. This should help is tackle more complex sequence-to-sequence problems.

Since the first output character of each target sequence has no previous character, we will need a new token to represent the start-of-sequence (sos).

During inference, we won't know the target, so what will we feed the decoder? We can just predict one character at a time, starting with an sos token, then feeding the decoder all the characters that were predicted so far (we will look at this in more details later in this notebook).

But if the decoder's LSTM expects to get the previous target as input at each step, how shall we pass it it the vector output by the encoder? Well, one option is to ignore the output vector, and instead use the encoder's LSTM state as the initial state of the decoder's LSTM (which requires that encoder's LSTM must have the same number of units as the decoder's LSTM).

Now let's create the decoder's inputs (for training, validation and testing). The sos token will be represented using the last possible output character's ID + 1.

In [45]:
OUTPUT_CHARS

'0123456789-'

In [44]:
sos_id = len(OUTPUT_CHARS) + 1
sos_id

12

In [47]:
def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

Let's take a look at the decoder's training inputs:

In [48]:
X_train_decoder

<tf.Tensor: shape=(10000, 10), dtype=int32, numpy=
array([[12,  8,  1, ..., 10, 11,  3],
       [12,  9,  6, ...,  6, 11,  2],
       [12,  8,  2, ...,  2, 11,  2],
       ...,
       [12, 10,  8, ...,  2, 11,  4],
       [12,  2,  2, ...,  3, 11,  3],
       [12,  8,  9, ...,  8, 11,  3]])>

Now let's build the model. It's not a simple sequential model anymore, so let's use the functional API:

In [49]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embedding = tf.keras.layers.Embedding(
    input_dim=len(INPUT_CHARS) + 1,
    output_dim=encoder_embedding_size)(encoder_input)
_, encoder_state_h, encoder_state_c = tf.keras.layers.LSTM(
    lstm_units, return_state=True)(encoder_embedding)
encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = tf.keras.layers.Input(shape=[None], dtype=tf.int32)
decoder_embedding = tf.keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2,
    output_dim=decoder_embedding_size)(decoder_input)
decoder_lstm_output = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state)
decoder_output = tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1,
                                    activation="softmax")(decoder_lstm_output)

model = tf.keras.Model(inputs=[encoder_input, decoder_input],
                           outputs=[decoder_output])

optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10,
                    validation_data=([X_valid, X_valid_decoder], Y_valid))

Epoch 1/10
313/313 [==============================] - 10s 20ms/step - loss: 1.6165 - accuracy: 0.3973 - val_loss: 1.3230 - val_accuracy: 0.5080
Epoch 2/10
313/313 [==============================] - 5s 17ms/step - loss: 1.0862 - accuracy: 0.6098 - val_loss: 0.8525 - val_accuracy: 0.7053
Epoch 3/10
313/313 [==============================] - 5s 16ms/step - loss: 0.6421 - accuracy: 0.7839 - val_loss: 0.4825 - val_accuracy: 0.8396
Epoch 4/10
313/313 [==============================] - 5s 16ms/step - loss: 0.2913 - accuracy: 0.9199 - val_loss: 0.2024 - val_accuracy: 0.9520
Epoch 5/10
313/313 [==============================] - 5s 17ms/step - loss: 0.1213 - accuracy: 0.9822 - val_loss: 0.0830 - val_accuracy: 0.9904
Epoch 6/10
313/313 [==============================] - 5s 17ms/step - loss: 0.0530 - accuracy: 0.9970 - val_loss: 0.0467 - val_accuracy: 0.9966
Epoch 7/10
313/313 [==============================] - 6s 18ms/step - loss: 0.0828 - accuracy: 0.9873 - val_loss: 0.0349 - val_accuracy: 0.998

Let's once again use the model to make some predictions. This time we need to predict characters one by one.

In [50]:
sos_id = len(OUTPUT_CHARS) + 1

def predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = tf.fill(dims=(len(X), 1), value=sos_id)
    for index in range(max_output_length):
        pad_size = max_output_length - Y_pred.shape[1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict([X, X_decoder])[:, index:index+1]
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32)
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=1)
    return ids_to_date_strs(Y_pred[:, 1:])

In [51]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

1/1 [==============================] - 0s 20ms/step


['1789-07-14', '2020-05-01']